In [1]:
%env KERAS_BACKEND = tensorflow
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import imdb
#num_words=10000意思接受出現頻率等於10000次的字
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=10000)

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


17465344/17464789 [==============================] - 10s 1us/step


In [5]:
#提供的training data和testing data數量
len(x_train)
len(x_test)

25000

In [6]:
#輸入資料的樣子，每個數字代表一個字，1為第一常出現，以此類推
x_train[99]

[1,
 1230,
 3765,
 566,
 97,
 189,
 102,
 86,
 7,
 32,
 4,
 973,
 16,
 55,
 355,
 18,
 14,
 20,
 4,
 64,
 542,
 173,
 16,
 4,
 893,
 2115,
 5376,
 250,
 39,
 8013,
 4,
 1362,
 2,
 14,
 102,
 47,
 57,
 599,
 633,
 6,
 1317,
 2,
 8,
 6,
 189,
 20,
 57,
 206,
 57,
 116,
 5,
 57,
 836,
 82,
 6,
 1317,
 2,
 3728,
 2,
 9,
 6,
 52,
 284,
 21,
 29,
 9,
 38,
 2245,
 5,
 1044,
 11,
 14,
 15,
 45,
 619,
 50,
 71,
 6,
 171,
 531,
 15,
 71,
 424,
 8,
 30,
 163,
 6211,
 4,
 1629,
 189,
 212,
 102,
 5,
 57,
 31,
 1498,
 11,
 4,
 311,
 13,
 197,
 15,
 14,
 20,
 16,
 1150,
 1479,
 5,
 13,
 161,
 990,
 692,
 5,
 1706,
 12,
 69,
 77,
 1194,
 8,
 3245,
 2001,
 553,
 67,
 14,
 20,
 48,
 25,
 423,
 13,
 131,
 124,
 51,
 25,
 122,
 236,
 1506,
 198,
 4,
 64,
 552,
 7,
 415,
 37,
 62,
 169,
 14,
 20,
 60,
 2602,
 629,
 5,
 615,
 14,
 9,
 8,
 25,
 1230,
 3765,
 570,
 231,
 189,
 102,
 14,
 20,
 166,
 2039,
 168,
 40,
 2450,
 5486,
 3298]

In [8]:
#前十筆資料的長度(字數)
for i in range(10):
    print(len(x_train[i]),end=',')

218,189,141,550,147,43,123,562,233,130,

In [9]:
#輸出資料的樣子，0(負評)，1(正評)
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

In [16]:
#設輸入資料長度的上限，並把每段文字弄成一樣長，太短的後面補 0
from keras.preprocessing import sequence
x_train = sequence.pad_sequences(x_train,maxlen=100)
x_test = sequence.pad_sequences(x_test,maxlen=100)
x_train.shape

(25000, 100)

In [23]:
#建構RNN神經網路，將文字壓到比較小的維度，且這些向量又代表某些意思(如兩向量角度小表相關程度大)，這叫word embedding
from keras.models import Sequential
from keras.layers import Dense,Embedding
from keras.layers import GRU

In [24]:
model = Sequential()
#embedding層可以把離散或整數的資料變得連續，Embedding(原資料數,壓縮後數量)
#可用LSTM或GRU(較快)訓練
model.add(Embedding(10000,128))
model.add(GRU(150))
model.add(Dense(1,activation='sigmoid'))

In [25]:
#計算 3*(128上層傳來input+150鄰居傳來+1bias)*150個 + (128+150+1)*150 = 167400
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
gru_1 (GRU)                  (None, 150)               125550    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 151       
Total params: 1,405,701
Trainable params: 1,405,701
Non-trainable params: 0
_________________________________________________________________


In [26]:
#組裝
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
#訓練
model.fit(x_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
25000/25000 [==============================] - 104s 4ms/step - loss: 0.4189 - acc: 0.8021
Epoch 2/15
25000/25000 [==============================] - 102s 4ms/step - loss: 0.2448 - acc: 0.9040
Epoch 3/15
25000/25000 [==============================] - 142s 6ms/step - loss: 0.1561 - acc: 0.9423
Epoch 4/15
25000/25000 [==============================] - 134s 5ms/step - loss: 0.0962 - acc: 0.9671
Epoch 5/15
25000/25000 [==============================] - 130s 5ms/step - loss: 0.0597 - acc: 0.9794
Epoch 6/15
25000/25000 [==============================] - 125s 5ms/step - loss: 0.0412 - acc: 0.9870
Epoch 7/15
25000/25000 [==============================] - 98s 4ms/step - loss: 0.0249 - acc: 0.9920
Epoch 8/15
25000/25000 [==============================] - 155s 6ms/step - loss: 0.0183 - acc: 0.9942
Epoch 9/15
25000/25000 [==============================] - 110s 4ms/step - loss: 0.0154 - acc: 0.9946
Epoch 10/15
25000/25000 [==============================] - 125s 5ms/step - loss: 0.0124 - ac

In [28]:
#檢視成果
score = model.evaluate(x_test,y_test)

25000/25000 [==============================] - 29s 1ms/step


In [29]:
#檢測資料的loss和檢測資料的正確率
print('loss',score[0])
print('acc:',score[1])

loss 1.18376077208519
acc: 0.82892


In [30]:
#將神經網路存起來
model_json=model.to_json()
open('imdb_model_architecyure.json','w').write(model_json)

1877

In [31]:
model.save_weights('indb_model_weights.h5')

In [32]:
#另一種存的方式
model.save('myrnn.h5')